# IMPORTS

In [ ]:
import numpy as np
import sklearn
import scipy as sci
import scipy.stats as ss
import scipy.optimize as opt
import matplotlib.pyplot as plt
import graphviz
import sys, math, os, copy, time, csv, itertools
import graphviz as gv
import covar
import networkx as nx
import pyBN as pybn
from sklearn.metrics import roc_curve, auc, confusion_matrix
from scipy.stats import norm, beta
from scipy.special import gamma
from itertools import combinations, permutations
from math import log, exp, log2
from operator import itemgetter, attrgetter
from inverse_covariance import ModelAverage, QuicGraphLassoEBIC, QuicGraphLasso, QuicGraphLassoCV,AdaptiveGraphLasso

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

## UTILS

In [ ]:
def cov2cor( A ):
    """
    covariance matrix to correlation matrix.
    """
    d = np.sqrt(A.diagonal())
    A = ((A.T/d).T)/d
    return A

def cov2pcor(cov):
#    cov = np.cov(X, rowvar=False)
    D, _ = cov.shape
    sigma = -np.linalg.pinv(cov)
    for i in range(D):
        sigma[i,i] = -1*sigma[i,i]
    pi_obs = cov2cor(sigma)
    return pi_obs

def pcor2cov(pi):
    pi = -pi
    for i in range(D):
        pi[i,i] = -1*pi[i,i]
    
    return np.linalg.inv(pi)

def pcor2prec(pi):
    pi = -pi
    for i in range(D):
        pi[i,i] = -1*pi[i,i]
    return pi

def prec2pcor(pi):
    pi = cov2cor(pi)
    return pi

def generate_graph(pi):
    D, _ = pi.shape
    G = nx.Graph()
    G.add_nodes_from(range(D))   
    
    for i in range(D):
        for j in range(i):
            if abs(pi[i, j]) > 1e-8:
                G.add_edge(i, j)
            else:
                pass
    return G



def generate_empty_graph(pi):
    D, _ = pi.shape
    G = nx.Graph()
    G.add_nodes_from(range(D))   
    return G

def calculate_entropy(X, steps):
    # establish ranges for X
    lo, hi, N = min(X), max(X) + 1e-8, X.shape[0]
    entropy = 0.0
    for i in range(steps):
        xl = (lo + (hi-lo)*(i*1.0/steps))
        xh = lo + (hi - lo)*( (i+1.0) / steps)
        tmp = ((xl <= X) & (X < xh)).sum()
        if tmp == 0:
            continue
        else:
            p = 1.0 * tmp / N
            entropy -= p * log2(p)
    return entropy
    
def calculate_joint_entropy(X, Y, steps):
    X_lo, X_hi, N = min(X), max(X) + 1e-8, X.shape[0]
    Y_lo, Y_hi = min(Y), max(Y) + 1e-8
    entropy = 0.0
    cnt = 0
    for i in range(steps):
        for j in range(steps):
            xl = X_lo + (X_hi - X_lo)*(i*1.0/steps)
            xh = X_lo + (X_hi - X_lo)*( (i+1.0) / steps)
            
            yl = Y_lo + (Y_hi - Y_lo)*(j*1.0/steps)
            yh = Y_lo + (Y_hi - Y_lo)*( (j+1.0) / steps)
        
            tmp = ((xl <= X) & (X < xh) & (yl <= Y) & (Y < yh)).sum()
            if tmp == 0:
                continue
            else:
                cnt += tmp
                p = 1.0 * tmp / N
                entropy -= p * log2(p)
    return entropy
    
def calculate_mutual_information(X, Y, steps=10):
    h_X = calculate_entropy(X, steps)
    h_Y = calculate_entropy(Y, steps)
    h_XY = calculate_joint_entropy(X, Y, steps)
    return h_X + h_Y - h_XY

def partial_corr(X, i, j, k):
    D = X.shape[1]
    idx = np.zeros(D, dtype=np.bool)
    for k_val in k:
        idx[k_val] = True
    
    if len(k) > 0:
        beta_i = sci.linalg.lstsq(X[:, idx], X[:, j])[0]
        beta_j = sci.linalg.lstsq(X[:, idx], X[:, i])[0]

        res_j = X[:, j] - X[:, idx].dot(beta_i)
        res_i = X[:, i] - X[:, idx].dot(beta_j)

        corr = ss.pearsonr(res_i, res_j)[0]
    else:
        corr = ss.pearsonr(X[:, i], X[:, j])[0]
    
    return corr
            
def fisher_test(X, i, j, k):
    N, _ = X.shape
    pcorr = partial_corr(X, i, j, k)
    if 1.0 - pcorr**2 < 1e-9:
        pval = 1e-10
    else:
        tij = pcorr * (N - len(k) - 2)**0.5 / ((1.0 - (pcorr**2))**0.5)
        pval = 2*ss.t.sf(abs(tij), N - len(k) - 2)
    return pval

## GAUSSIAN DATA SIMULATOR

In [ ]:
def simulate_correlation_matrix(num_nodes, frac_edges, mean=0):
    N = num_nodes
    mu_A = frac_edges
    eps = 10**-4
    cors = np.random.uniform(low=-1.0, high=1.0, size=(N, N))
    edges = np.random.choice(2, size=(N,N), p=(1.0 - frac_edges, frac_edges))
    edges = np.tril(edges, -1)
    edges = np.multiply(edges, cors)
    edges = edges + np.transpose(edges)
    abs_edges = np.absolute(edges)
    
    col_sums = np.sum(abs_edges, axis=0) # columnwise
    col_sums = np.maximum(col_sums, np.ones(D))
    col_sums = np.add(col_sums, np.array([eps]*N))
    col_sums = np.diag(col_sums)
    
    pi = col_sums + edges
    pi = cov2cor(pi)
    return pi

# Returns samples of data multivariate Gaussian distributed

def simulate_gaussian_data(num_samples, dim, frac_edges, pcor=None):
    N = num_samples
    D = dim
    
    if pcor is None:
        pcor = simulate_correlation_matrix(D, frac_edges)
    else:
        pass
    
    P = pcor2cov(pcor)
    sample_data = np.random.multivariate_normal(mean=np.array([0.0]*D),
                                                cov=P,
                                                size=N)
    
    return sample_data, pcor

def nx2graphviz(nxg, node_names=None):
    G = gv.Graph('G')
    for _, v in enumerate(list(nxg.nodes())):
        if node_names is None:
            G.node(str(v), str(v))
        else:
            G.node(node_names[v], node_names[v])
            
    for _, v in enumerate(list(nxg.edges())):
        i, j = v
        if node_names is None:
            i, j = str(i), str(j)
        else:
            i, j = node_names[i], node_names[j]
        G.edge(i, j)
    return G


## NETWORK METRIC CALCULATOR

In [ ]:
def generate_network_metrics(g_true, g_pred, time_diff=0.0):
    D = len(g_true.nodes())
    actual, pred = np.zeros((D, D)), np.zeros((D, D))
    for e in g_true.edges():
        i, j = e
        actual[i,j], actual[j,i] = 1, 1
    for e in g_pred.edges():
        i, j = e
        pred[i,j], pred[j,i] = 1, 1
    il = np.tril_indices(D, k=-1)
    actual, pred = actual[il], pred[il]
    
    cf = confusion_matrix(actual, pred)
    
    ### Metrics to be returned
    tn, fp, fn, tp = cf[0][0], cf[0][1], cf[1][0], cf[1][1]
    
    sens = tp / (tp + fn)
    spec = tn / (tn + fp)
    fpr = fp / (fp + tn)
    ppv = tp / (tp + fp)
    denominator = (tp+fp) * (tp + fn) * (tn + fp) * (tn+fn)
    if denominator < 1e-8:
        denominator = 1.0
    mcc = (tp * tn - fp * fn) / ( denominator )**0.5
    
    if sens < 1e-9 or ppv < 1e-9:
        f1_score = 0
    else:
        f1_score = 2.0 / ( (1.0/sens) + (1.0/ppv))
    
    hd = sum(abs(pred*1 - actual*1)) 
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return (tp, fp, tn, fn, sens, spec, fpr, ppv, mcc, f1_score, accuracy, hd, time_diff)

## RELEVANCE NETWORKS

In [ ]:
class RelevanceNetworks():
    
    def __init__(self, threshold=0.99):
        self._alpha = 0.99
        self.scores_ = None
        self.network_ = None
    
    def fit(self, sample_data):
        X = sample_data
        N, D = X.shape
        mutual_inf = np.zeros((D,D))
        
        for i in range(D):
            for j in range(i):
                mi = calculate_mutual_information(X[:, i], X[:, j], steps=int(log2(N)))
                mutual_inf[i, j], mutual_inf[j, i] = mi, mi

        # Calculate the MI threshold using permutations
        permutated_mi_vals = []
        for tt in range(1000):
            idx1 = np.random.randint(0, D)
            idx2 = np.random.randint(0, D)
            if idx1 == idx2:
                continue
            A, B = X[:, idx1], X[:, idx2]
            A = np.random.permutation(A)
            mi = calculate_mutual_information(A, B, steps=int(log2(N)))
            permutated_mi_vals.append(mi)

        permutated_mi_vals = sorted(permutated_mi_vals)
        threshold = permutated_mi_vals[int(len(permutated_mi_vals) * self._alpha)]
        
        gr_pred = nx.Graph()
        gr_pred.add_nodes_from(range(D))
        for i in range(D):
            for j in range(i):
                if mutual_inf[i, j] > threshold:
                    gr_pred.add_edge(i,j)
                else:
                    pass
        
        self.scores_ = mutual_inf
        self.network_ = gr_pred
        return

## FISHER APPROXIMATION

In [ ]:
class FisherApproximation():
    
    def __init__(self, threshold=0.95, bonf_correction=True):
        self._alpha = threshold
        self._use_bonf = bonf_correction
        self.scores_ = None
        self.network_ = None
    
    def fit(self, sample_data):
        X = sample_data
        N, D = X.shape    
        scores = np.zeros((D,D))
        cov = np.cov(X, rowvar=False)

        sigma = -np.linalg.inv(cov)
        for i in range(D):
            sigma[i,i] = -1*sigma[i,i]
        pi_obs = cov2cor(sigma)

        if self._use_bonf:
            n_tests = 0.5*D*(D-1)
            threshold = self._alpha / n_tests
        else:
            threshold = self._alpha
        
        gr_pred = nx.Graph()
        gr_pred.add_nodes_from(range(D))
        
        for i in range(D):
            for j in range(i):
                v = pi_obs[i,j]
                # Do the fisher transform
                zij = 0.5*log( (1+v) / (1-v) )
                zscore = (N - D + 2 - 3)**0.5 * abs(zij)
                scores[i,j] = 1 - 2*(1 - norm.cdf(zscore))
                scores[j,i] = scores[i,j]
                
                if norm.cdf(zscore) > 1 - 0.5*threshold:
                    gr_pred.add_edge(i,j)
                else:
                    pass
        self.scores_ = scores
        self.network_ = gr_pred
        return

## PC ALGORITHM

In [ ]:
class PCAlgorithm():
    
    def __init__(self, threshold=0.99, stable=True, init_graph=None):
        self._alpha = 1.0 - threshold
        self._stable = stable
        self.scores_ = None
        self.network_ = None
        self._init_graph = init_graph
        
    def fit(self, sample_data):
        X = sample_data
        N, D = X.shape
        node_ids = range(X.shape[1])

        if self._init_graph is None:
            g = nx.Graph()
            g.add_nodes_from(node_ids)
            for (i, j) in combinations(node_ids, 2):
                g.add_edge(i, j)
        else:
            g = self._init_graph
        
        l = 0
        max_reach = min(15, D-2)
        while True:
            cont = False
            remove_edges = []
            for (i, j) in permutations(node_ids, 2):
                adj_i = list(g.neighbors(i))
                if j not in adj_i:
                    continue
                else:
                    adj_i.remove(j)
                    pass
                if len(adj_i) >= l:
                    if len(adj_i) < l:
                        continue
                    for k in combinations(adj_i, l):
                        p_val = fisher_test(X, i, j, set(k))
                        if p_val > self._alpha:
                            if g.has_edge(i, j):
                                if self._stable:
                                    remove_edges.append((i, j))
                                else:
                                    g.remove_edge(i, j)
                                pass
                            break
                        pass
                    cont = True
                    pass
                pass
            l += 1
            if self._stable:
                g.remove_edges_from(remove_edges)
            if cont is False:
                break
            if l > max_reach:
                break
            pass

        self.network_ = g
        return 

## InterIAMB

In [ ]:
# Adapated from the pyBN repository 
def resolve_markov_blanket(Mb, data, alpha):
    n_rv = data.shape[1]
    
    Bd = copy.deepcopy(Mb)
    for X in range(n_rv):
        for Y in Mb[X]:
            if X not in Mb[Y]:
                Bd[X].remove(Y)
                #Bd[Y].append(X)
    
    for X in range(n_rv):
        for Y in Mb[X]:
            if (Y not in Bd[X] and X in Bd[Y]) or (X not in Bd[Y] and Y in Bd[X]):
                print("Neighbourhood boundary error!")
            elif (X not in Bd[Y]):
                # Edge already removed from graph
                continue
            else:
                pass
            
            if len(set(Bd[X]) - set([Y])) < len(set(Bd[Y]) - set([X])):
                T = list(set(Bd[X]) - set([Y]))
            else:
                T = list(set(Bd[Y]) - set([X]))
                
            for i in range(len(T)):
                removed_edge = False
                for S in itertools.combinations(T,i):
                    pval = fisher_test(data, X, Y, set(S))
                    if pval > alpha:
                        #print(X, Y, Bd[X], Mb[X], Bd[Y], Mb[Y])
                        Bd[X].remove(Y)
                        Bd[Y].remove(X)
                        removed_edge=True
                        break
                    else:
                        pass
                if removed_edge:
                    break

    edge_dict = dict([(rv,[]) for rv in range(n_rv)])
    for X in range(n_rv):
        for Y in Bd[X]:
            edge_dict[Y].append(X)
    
    return edge_dict

class InterIAMBAlgorithm():
    
    def __init__(self, threshold=0.99):
        self._alpha = 1.0 - threshold
        self.scores_ = None
        self.network_ = None

    def fit(self, sample_data):
        N, D = sample_data.shape
        node_ids = range(sample_data.shape[1])

        g = nx.Graph()
        g.add_nodes_from(node_ids)
        
        # Learn the markov blankets
        mb = dict([(node,[]) for node in node_ids])
        node_set = set(node_ids)
        
        for T in node_ids:
            # Phase 1 - Forward Phase
            CMB = set([])
            change = True
            while change:
                best = [1e8, -1]
                for X in node_ids:
                    if X != T and X not in CMB:
                        pv = fisher_test(sample_data, T, X, CMB)
                        if pv < best[0]:
                            best = [pv, X]
                pval, X = best
                if pval < self._alpha:
                    CMB.add(X)
                    # Phase 2 - Shrink Phase
                    to_remove = set([])
                    for Y in list(CMB):
                        pv = fisher_test(sample_data, T, Y, CMB - set([Y]))
                        if pv >= self._alpha:
                            CMB.remove(Y)
                else:
                    change = False
            
            mb[T] = list(CMB)

        # Resolve the markov blankets
        edge_dict = resolve_markov_blanket(mb, sample_data, alpha=self._alpha)
        for node in edge_dict:
            edges = edge_dict[node]
            for child in edges:
                g.add_edge(node, child)
        
        self.network_ = g
        return

## Graphical Lasso

In [ ]:
class GraphicalLasso():
    def __init__(self, threshold=0.0, use_ebic=False, use_adaptive=False, gamma=0.0):
        self._threshold = threshold
        self._use_ebic = use_ebic
        self._use_adaptive = use_adaptive
        self._gamma = gamma
        
        self.scores_ = None
        self.network_ = None
    
    def fit(self, X):
        N, D = X.shape
        g = nx.Graph()
        g.add_nodes_from(range(D))
        scores = np.zeros((D,D))
        
        if self._use_ebic:
            estimator = QuicGraphLassoEBIC(gamma=self._gamma, init_method='cov')
        else:
            estimator = QuicGraphLassoCV(init_method='cov')
            
        if self._use_adaptive:
            model = AdaptiveGraphLasso(estimator=estimator,  method='inverse')
            model.fit(X)
            pi = prec2pcor(model.estimator_.precision_)
        else:
            model = estimator
            model.fit(X)
            pi = prec2pcor(model.precision_)
       
        for i in range(D):
            for j in range(i):
                v = pi[i,j]
                scores[i,j], scores[j,i] = abs(v), abs(v)
                
                if abs(v) > self._threshold:
                    g.add_edge(i,j)
        
        self.scores_ = scores
        self.network_ = g
        return

## MIXTURE MODEL

In [ ]:
class MixtureModel():
    def __init__(self, fdr_rate=0.1):
        self._Q = fdr_rate
        self.scores_ = None
        self.network_ = None
        
    
    def neg_log_likelihood(self, x, *args):
        data = list(args[0])
        N = len(data)
        ll = 0.0
        eta, k = x[0], x[1]
        
        for i in range(N):
            p = data[i]
            #f0 = (1 - p*p)**(0.5*(k-3)) * gamma(k/2) / (math.pi * gamma(0.5*(k-1)))
            f0 = abs(p) * beta.pdf(p*p, 0.5, 0.5*(k-1))
            fA = 0.5 # Assume the alternative distribution ~ U[-1,1]        
            tmp = (1-eta) * f0 + eta * fA
            ll -= log(tmp)

        return ll

    def optimize_mixture_model(self, X):
        eta, k = 0.03, 100.0
        res = opt.minimize(self.neg_log_likelihood, 
                     x0=(eta, k), 
                     args=X,
                     bounds=((1e-6, 0.5), (10, None)),
                     options={'gtol': 1e-6, 'disp': False})
        return res.x


    def calculate_p_values(self, pi_arr, k):
        n, p_vals = len(pi_arr), []
        for i in range(n):
            r = pi_arr[i]
            t_val = r * math.sqrt((k-1) / (1 - r*r))
            p_val = 2 * (1 - ss.t.cdf(abs(t_val), k) )
            p_vals.append((r, p_val, i))
        return p_vals

    def fit(self, X):
        N, D = X.shape
        E = 0.5*D*(D-1)
        g = nx.Graph()
        g.add_nodes_from(range(D))
        scores = np.zeros((D,D))
        
        cov_shrink, _ = covar.cov_shrink_ss(X)
        pi = cov2pcor(cov_shrink)

        # Indices of the lower triangle
        il = np.tril_indices(D, k=-1)
        pi_elems = pi[il]

        # Fit the mixture model
        params = self.optimize_mixture_model(pi_elems)
        eta_est, k_est = params
        N_eff = k_est + D - 1

        # Calculate the p-values using t-test
        p_vals = self.calculate_p_values(pi_elems, k_est)
        p_vals = sorted(p_vals, key=itemgetter(1))

        # False Discovery Rate multiple testing (Benjamini and Hochberg, 1995)
        for i, v in enumerate(p_vals):
            r, p, idx = v
            
            i_idx, j_idx = il[0][idx], il[1][idx]
            scores[i_idx, j_idx], scores[j_idx, i_idx] = p, p
            
            if p <= ((i+1)/E) * self._Q :
                g.add_edge(i_idx, j_idx)
            else:
                break
        
        self.network_ = g
        self.scores_ = scores
        return

## RUN SIMULATIONS

In [ ]:
metric_names = ['sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd']
method_names = ['RN', 'PC', 'Mixture', 'GL-CV', 'GL-BIC', 'Empty']
num_tests, num_methods, num_metrics = 200, len(method_names), len(metric_names)

metrics = np.zeros((num_tests, num_methods, num_metrics))
params = np.zeros((num_tests, 3))
for tt in range(num_tests):
    D = np.random.randint(low=10, high=50)
    N = np.random.randint(low=10, high=200)
    ne = 2 + 2*np.random.rand()
    print('Test #%i: D=%i, N=%i, ne=%.2f' % (tt, D, N, ne))
    eta = ne / (D-1.0)
    params[tt,:] = [N, D, eta]
    
    sample_data, pcor = simulate_gaussian_data(N, D, eta)
    actual_network = generate_graph(pcor)
    empty_network = generate_empty_graph(pcor)
    metrics[tt, 5, :] = generate_network_metrics(actual_network, empty_network)
    
    rn_model = RelevanceNetworks()
    rn_model.fit(sample_data)
    metrics[tt, 0, :] = generate_network_metrics(actual_network, rn_model.network_)
    
    pc_model = PCAlgorithm()
    pc_model.fit(sample_data)
    metrics[tt, 1, :] = generate_network_metrics(actual_network, pc_model.network_)
    
    mix_model = MixtureModel()
    mix_model.fit(sample_data)
    metrics[tt, 2, :] = generate_network_metrics(actual_network, mix_model.network_)

    gl_model = GraphicalLasso()
    gl_model.fit(sample_data)
    metrics[tt, 3, :] = generate_network_metrics(actual_network, gl_model.network_)

    gl_model = GraphicalLasso(use_ebic=True, gamma=0.0)
    gl_model.fit(sample_data)
    metrics[tt, 4, :] = generate_network_metrics(actual_network, gl_model.network_)


In [ ]:
plot_metrics = np.mean(metrics, axis=0)
metric_names = ['sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd']
method_names = ['RN', 'PC', 'Mixture', 'GL-CV', 'GL-BIC', 'Empty']

metric_inds = [0, 3, 6, 7]
plt.figure(figsize=(12,12))
for i, idx in enumerate(metric_inds):
    plt.subplot(2,2,i+1)
    plt.boxplot(metrics[:50, :, idx])
    plt.title(metric_names[idx])
plt.show()

## PC / IAMB COMP

In [ ]:
metric_names = ['sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd']
method_names = ['PC', 'IAMB', 'Empty']
num_tests, num_methods, num_metrics = 50, len(method_names), len(metric_names)

metrics = np.zeros((num_tests, num_methods, num_metrics))
params = np.zeros((num_tests, 3))

times = np.zeros((num_tests,2))
                
for tt in range(num_tests):
    N = np.random.randint(low=10, high=25)
    D = np.random.randint(low=10, high=30)
    ne = 1 + 2*np.random.rand()
    print('Test #%i: D=%i, N=%i, ne=%.2f' % (tt, D, N, ne))
    eta = ne / (D-1.0)
    params[tt,:] = [N, D, eta]
    
    sample_data, pcor = simulate_gaussian_data(N, D, eta)
    actual_network = generate_graph(pcor)
    
    st = time.time()
    pc_model = PCAlgorithm()
    pc_model.fit(sample_data)
    metrics[tt, 1, :] = generate_network_metrics(actual_network, pc_model.network_)   
    times[tt, 1] = time.time()-st
    
    iamb_model = InterIAMBAlgorithm()
    iamb_model.fit(sample_data)
    metrics[tt, 2, :] = generate_network_metrics(actual_network, iamb_model.network_)

In [ ]:
metric_names = ['sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd']
method_names = ['PC', 'IAMB', 'Empty']

metric_inds = [0, 3, 6, 7]
plt.figure(figsize=(12,12))
for i, idx in enumerate(metric_inds):
    plt.subplot(2,2,i+1)
    plt.boxplot(metrics[:, :, idx])
    plt.title(metric_names[idx])
plt.show()

## Read in the E Coli data

In [ ]:
# TRANSFORM THE MODEL STRING
def modelstring2graph(model_string, node_dict):
    D = len(node_dict)
    adj = np.zeros((D,D))
    x = model_string
    nodes = x[1:-1].split('][')
    for v in nodes:
        if "|" not in v:
            continue
        else:
            child, parent_string = v.split('|')
            parents = parent_string.split(':')
            child_idx = node_dict[child]
            for p in parents:
                parent_idx = node_dict[p]
                adj[child_idx][parent_idx] = 1.0
                adj[parent_idx][child_idx] = 1.0
    return adj

fn = '/mnt/c/Users/Chris/Documents/Dissertation/Code/R/ecoli_continous_sample_1.csv'
ecoli_data = np.genfromtxt(fn, delimiter=',', skip_header=1)
nodes = ["aceB","asnA","atpD","atpG","b1191","b1583","b1963","cchB","cspA","cspG","dnaG","dnaJ","dnaK","eutG","fixC","flgD","folK","ftsJ","gltA","hupB","ibpB","icdA","lacA","lacY","lacZ","lpdA","mopB","nmpC","nuoM","pspA","pspB","sucA","sucD","tnaA","yaeM","yceP","ycgX","yecO","yedE","yfaD","yfiA","ygbD","ygcE","yhdM","yheI","yjbO"]
node_dict = {}
modelstring = "[b1191][cspG][eutG][cspA|cspG][fixC|b1191][sucA|eutG][yecO|cspG][yedE|cspG][atpG|sucA][cchB|fixC][dnaJ|sucA][flgD|sucA][gltA|sucA][lpdA|yedE][pspB|cspG:yedE][sucD|sucA][tnaA|b1191:fixC:sucA][yceP|eutG:fixC][yfiA|cspA][ygbD|fixC][ygcE|b1191:sucA][yhdM|sucA][yjbO|fixC][asnA|ygcE][atpD|sucA:ygcE][hupB|cspA:yfiA][ibpB|eutG:yceP][pspA|cspG:pspB:yedE][yfaD|eutG:sucA:yceP][icdA|asnA:ygcE][lacA|asnA:cspG][nmpC|pspA][yheI|atpD:yedE][aceB|icdA][b1963|yheI][dnaK|yheI][folK|yheI][lacY|asnA:cspG:eutG:lacA][ycgX|fixC:yheI][dnaG|ycgX:yheI][lacZ|asnA:lacA:lacY][nuoM|lacY][b1583|lacA:lacZ:yceP][mopB|dnaK:lacZ][yaeM|cspG:lacA:lacZ][ftsJ|mopB]"
for i, v in enumerate(nodes):
    node_dict[v] = i

ecoli_adj = modelstring2graph(modelstring, node_dict)

## MAGIC-IRRI DATA

In [ ]:
nodes = ["YR.GLASS","HT","YR.FIELD","MIL","FT","G418","G311","G1217","G800","G866","G795","G2570","G260","G2920","G832","G1896","G2953","G266","G847","G942","G200","G257","G2208","G1373","G599","G261","G383","G1853","G1033","G1945","G1338","G1276","G1263","G1789","G2318","G1294","G1800","YLD","FUS","G1750","G524","G775","G2835","G43"]
node_dict = {}
modelstring="[G418][G1217][G800][G866][G795][G2570][G260][G2920][G832][G2953][G847][G942][G200][G599][G261][G1853][G1033][G1945][G1338][G1263][G2318][G1750][G524][G775][G311|G1853][G1896|G2953][G257|G1217][G1373|G1750][G383|G800][G1276|G599][G1294|G418][G2835|G418][G266|G1338:G1276][G2208|G257][G1800|G1217:G2953:G257:G2835][G43|G311][HT|G832:G1896:G2953:G266:G847:G942:G2835][MIL|G1217:G2208:G1945:G1338:G524][FT|G266:G1276:G1263:G2318:G1294:G1800:G775][G1789|G266][YR.GLASS|MIL:G418:G311:G1217:G800:G866:G795:G1750][FUS|HT:G832:G1896:G383:G1853:G1033][YR.FIELD|YR.GLASS:FT:G418:G200:G257:G2208:G1373:G599:G261][YLD|YR.GLASS:HT:FT:G2570:G260:G2920:G832]"
for i, v in enumerate(nodes):
    node_dict[v] = i

niab_adj = modelstring2graph(modelstring, node_dict)

## ARTH DATA

In [ ]:
nodes = ["4","8","13","20","26","38","47","61","63","78","81","86","93","96","100","101","108","111","126","135","144","155","161","181","187","197","198","209","211","219","226","234","245","248","256","269","272","281","289","296","299","328","331","342","360","363","368","377","378","412","414","422","443","444","452","454","460","464","479","480","496","519","537","539","540","547","554","558","560","565","570","573","576","585","596","598","600","603","622","623","627","629","636","640","651","661","665","666","677","679","686","699","712","714","726","736","738","739","758","767","778","779","781","783","786","793","798"]
node_dict = {}
modelstring ="[81][100][414][422][519][738][783][86|738][144|81][187|783][198|81][209|738][211|81][245|100][248|81][296|100][328|519][368|81][377|81][412|100][454|783][540|738][547|783][558|783][570|81:422][622|81][627|422:738][629|422][636|81:738][640|783][665|81][666|81][679|783][712|81][726|414][781|783][8|209][20|781][26|781][38|570][61|81:570][63|198:738][78|726][96|558][126|558:783][135|570][155|679][161|558][181|86:783][219|570][234|570][256|558:570][269|558:783][281|627][342|558][363|558][378|570][460|570][464|570][554|570][576|570][585|570][598|570][603|781][623|570][651|570][661|627][686|198][699|570][739|558][767|81:570][779|198][108|81:603][111|61:81:570][444|63:738][452|414:603:726][480|61:738][539|585][600|699][798|779][4|539][93|539][197|539][272|108:414:452:603:726][299|444][360|539][443|600][496|111][537|111][596|539][758|539][778|539][101|443][289|272:414:452:726][560|272:627:726][565|443][573|20:443][226|573][786|289:414:726][793|81:560:629][47|414:422:452:629:793][479|47:422:629:793][736|47:422][331|47:414:422:452:479][13|331:422][714|47:331:422:452:479:629][677|479:714]"
for i, v in enumerate(nodes):
    node_dict[v] = i

arth_adj = modelstring2graph(modelstring, node_dict)

## RUN THE EXPERIMENTS

In [ ]:
metric_names = ['tp', 'fp', 'tn', 'fn', 'sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd', 'runtime']
method_names = ['RN', 'PC', 'InterIAMB', 'Mixture', 'Adaptive GL-CV3', 'Adaptive GL-BIC', 'Empty']
num_tests, num_methods, num_metrics = 50, len(method_names), len(metric_names)

metrics = np.zeros((num_tests, num_methods, num_metrics))

actual_network = generate_graph(ecoli_adj)
empty_network = generate_empty_graph(ecoli_adj)

for tt in range(num_tests):    
    #fn = ('/mnt/c/Users/Chris/Documents/Dissertation/Code/R/FINAL_SMALL_ECOLI_sample_%i.csv' % (tt+1))
    #ecoli_data = np.genfromtxt(fn, delimiter=',', skip_header=1)
    #ecoli_data = ecoli_data - ecoli_data.mean(axis=0, keepdims=True)
    #sample_data = ecoli_data
    print("Now running test #%i" % (tt+1))
    D = 50
    N = 500
    ne = 3
    eta = ne / (D-1.0)
    num_edges = -1
    while(num_edges != 70):
        sample_data, pcor = simulate_gaussian_data(N, D, eta)
        actual_network = generate_graph(pcor)
        empty_network = generate_empty_graph(pcor)
        num_edges = generate_network_metrics(actual_network, empty_network, time.time()-st)[-2]
    #st = time.time()
    #rn_model = RelevanceNetworks()
    #rn_model.fit(sample_data)
    #metrics[tt, 0, :] = generate_network_metrics(actual_network, rn_model.network_, time.time()-st)

    st = time.time()
    pc_model = PCAlgorithm(threshold=0.99)
    pc_model.fit(sample_data)
    metrics[tt, 1, :] = generate_network_metrics(actual_network, pc_model.network_, time.time()-st)
    st = time.time()
    iamb_model = InterIAMBAlgorithm(threshold=0.99)
    iamb_model.fit(sample_data)
    metrics[tt, 2, :] = generate_network_metrics(actual_network, iamb_model.network_, time.time()-st)

    st = time.time()
    mix_model = MixtureModel(fdr_rate=0.1)
    mix_model.fit(sample_data)
    metrics[tt, 3, :] = generate_network_metrics(actual_network, mix_model.network_, time.time()-st)
    
    st = time.time()
    gl_model = GraphicalLasso(use_ebic=False, use_adaptive=True)
    gl_model.fit(sample_data)
    metrics[tt, 4, :] = generate_network_metrics(actual_network, gl_model.network_, time.time()-st)
    
    st = time.time()
    gl_model = GraphicalLasso(use_ebic=True, use_adaptive=True, gamma=0.0)
    gl_model.fit(sample_data)
    metrics[tt, 5, :] = generate_network_metrics(actual_network, gl_model.network_, time.time()-st)

    st = time.time()
    metrics[tt, 6, :] = generate_network_metrics(actual_network, empty_network, time.time()-st)


## PCOR Calculator

### SIG LEVEL FOR VARYING N

In [ ]:
metric_names = ['tp', 'fp', 'tn', 'fn', 'sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd', 'runtime']
method_names = ['RN', 'PC', 'InterIAMB', 'Mixture', 'Adapative GL-CV3', 'GL-BIC', 'Empty']
num_tests, num_methods, num_metrics = 50, len(method_names), len(metric_names)

sig_levels = [[1.0-1e-6, 1.0-1e-5, 1.0-1e-4, 1.0-1e-3, 0.99, 0.975, 0.95, 0.9, 0.8],
              [1.0-1e-7, 1.0-1e-6, 1.0-1e-5, 1.0-1e-4, 1.0-1e-3, 0.99, 0.975, 0.95, 0.9],
              [0.0001, 0.001, 0.01, 0.02, 0.05,  0.1,  0.25, 0.5, 1.0]]

N_vals = [10, 25, 50, 200, 1000]    
#roc_curves = np.zeros((num_tests, 3, len(sig_levels[0]), len(N_vals)))
for tt in range(num_tests):    
    print("Now running test #%i" % (tt+1))
    D = 50
    ne = 3
    eta = ne / (D-1.0)
    num_edges = -1
    while(num_edges != 70):
        sample_data, pcor = simulate_gaussian_data(100, D, eta)
        actual_network = generate_graph(pcor)
        empty_network = generate_empty_graph(pcor)
        num_edges = generate_network_metrics(actual_network, empty_network, time.time()-st)[-2]
    
    for i, N in enumerate(N_vals):
        if i == 0:
            continue
        sample_data, _ = simulate_gaussian_data(N, D, -1, pcor=pcor)
#        for idx, alpha in enumerate(sig_levels[0]):
#            print("\t PC: alpha=%.7f" % alpha)
#            pc_model = PCAlgorithm(threshold=alpha)
#            pc_model.fit(sample_data)
#            roc_curves[tt, 0, idx, i] = generate_network_metrics(actual_network, pc_model.network_)[-2]
        
        for idx, alpha in enumerate(sig_levels[1]):
            # print("\t IAMB: alpha=%.7f" % alpha)
            iamb_model = InterIAMBAlgorithm(threshold=alpha)
            iamb_model.fit(sample_data)
            roc_curves[tt, 1, idx, i] = generate_network_metrics(actual_network, iamb_model.network_)[-2]
        '''
        for idx, alpha in enumerate(sig_levels[2]):
            print("\t FDT: Q=%.7f" % alpha)
            mix_model = MixtureModel(fdr_rate=alpha)
            mix_model.fit(sample_data)
            roc_curves[tt, 2, idx, i] = generate_network_metrics(actual_network, mix_model.network_)[-2]
        '''

In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
#plt.title('Effect of significance size for varying sample size', fontsize=20)
labels = ['PC', 'InterIAMB', 'Mixture']
for idx in range(len(N_vals)):
    if idx == 0:
        continue
    x = 1-np.array(sig_levels[1])
    tmp = roc_curves[:, 1, :, idx]
    y = np.nanmean(tmp[:, :], axis=0)
    yerr = np.nanstd(tmp[:, :], axis=0) / ((50**0.5) /1.96)
#    plt.plot(x, y, 'o--', label=labels[idx])
    plt.errorbar(x, y, yerr, xerr=None, fmt='o--', label=("N=%i" % N_vals[idx]))
plt.xlabel('1 - Sig. Level', fontsize=20)
plt.ylabel('Hamming Distance', fontsize=20)
plt.xscale('log')
#plt.xlim([0,1])
plt.title('InterIAMB Algorithm', fontsize=20)
plt.legend(fontsize=14, loc='lower left')
plt.suptitle('Hamming Distance of simulated Gaussian inference for varying sample sizes and parameters', fontsize=20)
plt.show()



## ROC Curve generation

In [ ]:
metric_names = ['tp', 'fp', 'tn', 'fn', 'sens', 'spec', 'fpr', 'ppv', 'mcc', 'f1_score', 'accuracy', 'hd', 'runtime']
method_names = ['RN', 'PC', 'InterIAMB', 'Mixture', 'Adapative GL-CV3', 'GL-BIC', 'Empty']
num_tests, num_methods, num_metrics = 50, len(method_names), len(metric_names)

sig_levels = [[1.0-1e-6, 1.0-1e-5, 1.0-1e-4, 1.0-1e-3, 0.99, 0.975, 0.95, 0.9, 0.8],
              [1.0-1e-7, 1.0-1e-6, 1.0-1e-5, 1.0-1e-4, 1.0-1e-3, 0.99, 0.975, 0.95, 0.9],
              [0.0001, 0.001, 0.01, 0.02, 0.05,  0.1,  0.25, 0.5, 1.0]]

roc_curves = np.zeros((num_tests, 3, len(sig_levels[0]), num_metrics))
    

actual_network = generate_graph(arth_adj)
empty_network = generate_empty_graph(arth_adj)

for tt in range(num_tests):    
    #fn = ('/mnt/c/Users/Chris/Documents/Dissertation/Code/R/FINAL_LARGE_ARTH_sample_%i.csv' % (tt+1))
    #ecoli_data = np.genfromtxt(fn, delimiter=',', skip_header=1)
    #ecoli_data = ecoli_data - ecoli_data.mean(axis=0, keepdims=True)
    #sample_data = ecoli_data
    print("Now running test #%i" % (tt+1))
    D = 50
    N = 500
    ne = 3
    eta = ne / (D-1.0)
    num_edges = -1
    while(num_edges != 70):
        sample_data, pcor = simulate_gaussian_data(N, D, eta)
        actual_network = generate_graph(pcor)
        empty_network = generate_empty_graph(pcor)
        num_edges = generate_network_metrics(actual_network, empty_network, time.time()-st)[-2]
    
    for idx, alpha in enumerate(sig_levels[0]):
        print("\t PC: alpha=%.7f" % alpha)
        pc_model = PCAlgorithm(threshold=alpha)
        pc_model.fit(sample_data)
        roc_curves[tt, 0, idx, :] = generate_network_metrics(actual_network, pc_model.network_)
    for idx, alpha in enumerate(sig_levels[1]):
        print("\t IAMB: alpha=%.7f" % alpha)
        iamb_model = InterIAMBAlgorithm(threshold=alpha)
        iamb_model.fit(sample_data)
        roc_curves[tt, 1, idx, :] = generate_network_metrics(actual_network, iamb_model.network_)
    for idx, alpha in enumerate(sig_levels[2]):
        print("\t FDT: Q=%.7f" % alpha)
        mix_model = MixtureModel(fdr_rate=alpha)
        mix_model.fit(sample_data)
        roc_curves[tt, 2, idx, :] = generate_network_metrics(actual_network, mix_model.network_)

In [ ]:
print(sig_levels[2])
np.nanmean(roc_curves[:, 2, :, 11], axis=0)


In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
plt.title('Precision / Recall', fontsize=20)
labels = ['PC', 'InterIAMB', 'Mixture']
for idx in range(3):
    x = np.nanmean(roc_curves[:, idx, :, 4], axis=0)
    y = np.nanmean(roc_curves[:, idx, :, 7], axis=0)
    yerr = np.nanstd(roc_curves[:, idx, :, 7], axis=0) / ((50**0.5) /1.96)
#    x, y = np.zeros((11, 1)), np.zeros((11, 1))
#    x[-1, 0], y[0, 0] = 1.0, 1.0
#    x[1:-1, 0] = np.mean(roc_curves[:, idx, :, 4], axis=0)
#    y[1:-1, 0] = np.mean(roc_curves[:, idx, :, 7], axis=0)
#    plt.plot(x, y, 'o--', label=labels[idx])
    plt.errorbar(x, y, yerr, xerr=None, fmt='o--', label=labels[idx])
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.xlim([0,1])
plt.ylim([0,1])
plt.legend(fontsize=14, loc='lower left')

plt.subplot(1,2,2)
plt.title('Hamming Distance / Graph Size', fontsize=20)
labels = ['PC', 'InterIAMB', 'Mixture']
for idx in range(3):
    #x, y = np.zeros((18, 1)), np.zeros((18, 1))
    #x[17, 0], y[0, 0] = 1.0, 1.0
#    x = np.mean(roc_curves[:, idx, :, 0], axis=0) + np.mean(roc_curves[:, idx, :, 1], axis=0)
#    y = np.mean(roc_curves[:, idx, :, 11], axis=0)
#    plt.plot(x, y, 'o--', label=labels[idx])
    x = np.mean(roc_curves[:, idx, :, 0], axis=0) + np.mean(roc_curves[:, idx, :, 1], axis=0)
    y = np.mean(roc_curves[:, idx, :, 11], axis=0)
    yerr = np.std(roc_curves[:, idx, :, 11], axis=0) / ((50**0.5)/1.96)
#    x, y = np.zeros((11, 1)), np.zeros((11, 1))
#    x[-1, 0], y[0, 0] = 1.0, 1.0
#    x[1:-1, 0] = np.mean(roc_curves[:, idx, :, 4], axis=0)
#    y[1:-1, 0] = np.mean(roc_curves[:, idx, :, 7], axis=0)
#    plt.plot(x, y, 'o--', label=labels[idx])
    plt.errorbar(x, y, yerr, xerr=None, fmt='o--', label=labels[idx])
plt.xlabel('Number of edges in predicted network', fontsize=20)
plt.ylabel('Hamming Distnace', fontsize=20)
plt.xlim([40, 100])
plt.ylim([0, 40])
plt.legend(fontsize=14, loc='lower right')
plt.suptitle('Simulated undirected Gaussian model inference, D=50, N=500', fontsize=20)
plt.show()



In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
plt.title('ROC Curve for small sample Gaussian network inference', fontsize=16)
labels = ['PC', 'InterIAMB', 'Mixture']
for idx in range(3):
    x, y = np.zeros((11, 1)), np.zeros((11, 1))
    x[-1, -1], y[-1, -1] = 1.0, 1.0
    x[1:-1, 0] = np.mean(roc_curves[:, idx, :, 4], axis=0)
    y[1:-1, 0] = 1-np.mean(roc_curves[:, idx, :, 5], axis=0)
    plt.plot(y, x, 'o--', label=labels[idx])
plt.plot([0,1],[0,1], '-.')
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.xlim([0,1.0])
plt.ylim([0.0,1])
plt.legend(fontsize=14, loc='lower right')
plt.show()

## Alpha Impact on PC for ECOLI

In [ ]:
num_tests = 50
pc_sig_levels = [0.9999999, 0.999999, 0.99999, 0.9999, 0.999, 0.995, 0.99, 0.975, 0.95, 0.9, 0.8, 0.5]
pc_alpha_metrics = np.zeros((num_tests, len(pc_sig_levels), 2))
actual_network = generate_graph(ecoli_adj)
empty_network = generate_empty_graph(ecoli_adj)

for tt in range(num_tests):    
    fn = ('/mnt/c/Users/Chris/Documents/Dissertation/Code/R/SMALL_ecoli_continous_sample_%i.csv' % (tt+1))
    ecoli_data = np.genfromtxt(fn, delimiter=',', skip_header=1)
    ecoli_data = ecoli_data - ecoli_data.mean(axis=0, keepdims=True)
    sample_data = ecoli_data
    print("Now running test #%i" % (tt+1))
    for i, alpha in enumerate(pc_sig_levels):
        print("\t PC: alpha=%.6f" % alpha)
        pc_model = PCAlgorithm(threshold=alpha)
        pc_model.fit(sample_data)
        stats = generate_network_metrics(actual_network, pc_model.network_)
        mcc, hd = stats[8], stats[11]
        pc_alpha_metrics[tt, i, :] = [mcc, hd]


iamb_sig_levels = [0.9999999, 0.999999, 0.99999, 0.9999, 0.999, 0.995, 0.99, 0.975, 0.95, 0.9]
iamb_alpha_metrics = np.zeros((num_tests, len(iamb_sig_levels), 2))
for tt in range(num_tests):    
    fn = ('/mnt/c/Users/Chris/Documents/Dissertation/Code/R/SMALL_ecoli_continous_sample_%i.csv' % (tt+1))
    ecoli_data = np.genfromtxt(fn, delimiter=',', skip_header=1)
    ecoli_data = ecoli_data - ecoli_data.mean(axis=0, keepdims=True)
    sample_data = ecoli_data
    print("Now running test #%i" % (tt+1))
    for i, alpha in enumerate(iamb_sig_levels):
        mcc, hd = stats[8], stats[11]
        
        print("\t IAMB: alpha=%.6f" % alpha)
        iamb_model = InterIAMBAlgorithm(threshold=alpha)
        iamb_model.fit(sample_data)
        stats = generate_network_metrics(actual_network, iamb_model.network_)
        mcc, hd = stats[8], stats[11]
        iamb_alpha_metrics[tt, i, :] = [mcc, hd]

titles = ['MCC', 'Hamming Distance']
alphas = list(map(lambda x: 1-x, iamb_sig_levels))
plt.figure(figsize=(12,6))
for i, idx in enumerate(titles):
    plt.subplot(1,2,i+1)
    alphas = list(map(lambda x: 1-x, iamb_sig_levels))
    plt.plot(alphas, np.mean(iamb_alpha_metrics[:, :, i], axis=0), 'o--', label='InterIAMB')
    alphas = list(map(lambda x: 1-x, pc_sig_levels))
    plt.plot(alphas, np.mean(pc_alpha_metrics[:, :, i], axis=0), 'o--', label='PC')
    plt.legend()
    plt.ylabel(titles[i], fontsize=16)
    plt.title(titles[i], fontsize=16)
    plt.tick_params(labelsize=12)
    if i == 0:
        plt.ylim([0.5,0.65])
    plt.xlabel('1-Sig Level', fontsize=16)
    plt.xscale('log')
plt.show()